<a href="https://colab.research.google.com/github/CloseChoice/easyshap/blob/main/shap_random_forest_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap

In [111]:
import numpy as np
import pandas as pd
import shap
import os
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import seaborn as sns

1. Load Dataset & Preprocessing

In [145]:
data = sns.load_dataset("titanic")
data = data.drop(['alive'], axis=1)

# Convert categorical variables into dummy/indicator variables
data_processed = pd.get_dummies(data)

# Filling Null Values
data_processed = data_processed.fillna(data_processed.mean())

X_train, X_test = train_test_split(data_processed, test_size=0.3, random_state=0, stratify=data_processed['survived'])

# Create X_train,Y_train,X_test
Y_train = X_train['survived']
Y_test = X_test['survived']
X_train = X_train.drop(['survived'], axis=1)
X_test = X_test.drop(['survived'], axis=1)

features = X_train.columns
print("features", features)

features Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'adult_male', 'alone',
       'sex_female', 'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S',
       'class_First', 'class_Second', 'class_Third', 'who_child', 'who_man',
       'who_woman', 'deck_A', 'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F',
       'deck_G', 'embark_town_Cherbourg', 'embark_town_Queenstown',
       'embark_town_Southampton'],
      dtype='object')


2. Fit Models

In [146]:
####################
# 1. Random Forest
####################
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
random_forest_preds = random_forest.predict(X_test)
print('1. The accuracy of the Random Forests model is :\t', metrics.accuracy_score(random_forest_preds, Y_test))

####################
# 2. XG Boost
####################
xgb = XGBClassifier(use_label_encoder=False)
xgb.fit(X_train, Y_train) # , eval_metric='rmse'
xgb_preds = xgb.predict(X_test)
print('2. The accuracy of the XG-Boost model is :\t', metrics.accuracy_score(xgb_preds, Y_test))

####################
# 3. Log Regression
####################
lg = LogisticRegression(solver='lbfgs', max_iter=1000)
lg.fit(X_train, Y_train)
lg_preds = lg.predict(X_test)
print('3. The accuracy of the Log-Regression model is :\t', metrics.accuracy_score(lg_preds, Y_test))

####################
# 4. XGB with Random Search
####################
gbm_param_grid = {
    'n_estimators': range(8, 20),
    'max_depth': range(6, 10),
    'learning_rate': [.4, .45, .5, .55, .6],
    'colsample_bytree': [.6, .7, .8, .9, 1]
}
gbm = XGBClassifier(n_estimators=10, use_label_encoder=False)
xgb_random = RandomizedSearchCV(param_distributions=gbm_param_grid,
                                estimator=gbm, scoring="accuracy",
                                verbose=1, n_iter=50, cv=4)
xgb_random.fit(X_train, Y_train) # , eval_metric='rmse'
xgb_random_preds = lg.predict(X_test)
print('4. The accuracy of the XGB with Random Search model is :\t', metrics.accuracy_score(xgb_random_preds, Y_test))

1. The accuracy of the Random Forests model is :	 0.8171641791044776
2. The accuracy of the XG-Boost model is :	 0.8097014925373134
3. The accuracy of the Log-Regression model is :	 0.8395522388059702
Fitting 4 folds for each of 50 candidates, totalling 200 fits
4. The accuracy of the XGB with Random Search model is :	 0.8395522388059702


3. Explain Models

**Feature Importance without shap**

In [147]:
importances = random_forest.feature_importances_
df = pd.DataFrame({'feature':features, 'importance':importances})
df = df.sort_values(by='importance', ascending = True)
df['importance'] = df['importance']*100
fig = px.bar(df, x='importance', y='feature', height=800, width=700)
fig = fig.update_layout(xaxis={'categoryorder':'category ascending'})
fig.show()

**Feature Importance with shap**

In [148]:
def explain_features(model, X, features, title='this'):
  print(f"Calculate Shap Values for {title} model")

  # Create Tree Explainer object that can calculate shap values
  if title == 'Logistic Regression':
    explainer = shap.Explainer(lg, X)
  else:
    explainer = shap.TreeExplainer(model)

  # Calculate Shap values
  if title == 'Random Forest':
    shap_values = explainer.shap_values(X)[0] # TODO: Why? I have no idea ..
  else:
    shap_values = explainer.shap_values(X)
  df_shap_values = pd.DataFrame(shap_values, columns=features)

  abs_sum = np.nansum(np.abs(shap_values), dtype=np.float64)
  print("abs_sum", abs_sum)

  print("observations", len(shap_values))

  df_shap_values = pd.DataFrame(shap_values, columns=features)
  df_shap_values = df_shap_values.agg([sum, np.mean, lambda x : sum(x.abs()), lambda x : sum(np.where(x<0, 0, x)), lambda x : sum(np.where(x>0, 0, x*-1))])
  df_shap_values.index = ["sum","mean","abs_sum", "sum_pos", "sum_neg"]

  df_shap_values = df_shap_values.transpose()
  df_shap_values['importance'] = (df_shap_values['abs_sum'] / abs_sum) * 100
  df_shap_values['importance_pos'] = (df_shap_values['sum_pos'] / abs_sum) * 100
  df_shap_values['importance_neg'] = (df_shap_values['sum_neg'] / abs_sum) * 100
  df_shap_values = df_shap_values.reset_index().rename({'index':'feature'}, axis = 'columns')
  return df_shap_values

def create_feature_plot(df, metric):
  df = df.sort_values(by=metric, ascending = True)
  fig = px.bar(df, x=metric, y='feature', height=800, width=700)
  fig.show()

**1. Random Forest Model - Shap Feature Importance**

In [156]:

df_rf = explain_features(random_forest, X_test, features, 'Random Forest')
create_feature_plot(df_rf, 'importance')

Calculate Shap Values for Random Forest model
abs_sum 141.327244829314
observations 268


**2. XG-Boost Model - Shap Feature Importance**

In [164]:
df_xgb = explain_features(xgb, X_test, features, 'XG-Boost')
create_feature_plot(df_xgb, 'importance')

Calculate Shap Values for XG-Boost model


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[14:29:14] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.



abs_sum 1489.354383988306
observations 268


**3. Logistic Regression - Shap Feature Importance**

In [174]:
df_lg = explain_features(lg, X_test, features, 'Logistic Regression')
create_feature_plot(df_lg, 'importance')

Calculate Shap Values for Logistic Regression model
abs_sum 948.0341340819906
observations 268


**4. XG-Boost with Random Search - Shap Feature Importance**

In [163]:
df_gbm = explain_features(xgb_random.best_estimator_, X_test, features, 'XG-Boost with Random Search')
create_feature_plot(df_gbm, 'importance')

Calculate Shap Values for XG-Boost with Random Search model
abs_sum 808.2889501914615
observations 268


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[14:29:09] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.



**Compare Importance of two Models**

In [177]:
def compare(df_1, df_2, column):
  df_1['diff'] = df_1[column]-df_2[column]
  create_feature_plot(df_1, 'diff')

compare(df_rf, df_lg, 'importance')

In [ ]:



# other plot
df = df_shap_values.copy()

fig = go.Figure()
fig.add_trace(go.Bar(x=-df['importance_pos'].values,
                        y=df['feature'],
                        orientation='h',
                        name='Male',
                        customdata=df['mean'],
                        hovertemplate = "Feature: %{y}<br>Pop:%{customdata}<br>Positive Contribution in %<extra></extra>"))
fig.add_trace(go.Bar(x= df['importance_neg'],
                        y =df['feature'],
                        orientation='h',
                        name='Female',
                        hovertemplate="Feature: %{y}<br>Pop:%{x}<br>Negative Contribution in %<extra></extra>"))

fig.update_layout(barmode='relative',
                  height=800,
                  width=700,
                  yaxis_autorange='reversed',
                  bargap=0.01,
                  legend_orientation ='h',
                  legend_x=-0.05, legend_y=1.1
                 )
fig.show()
